# Convert — Run a simple conversion of a PDB to the QDXF format

This notebook shows how to perform a simple conversion of PDBs to the QDXF format

# 0) Code Sample
See the [sample notebook](Notebooks/convert-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [1]:
import os
from glob import glob
from pathlib import Path

from pdbtools import pdb_fetch

import rush

## 1.1) Configuration

In [2]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "convert"]
WORK_DIR = Path.home() / "qdx" / "convert"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

In [3]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

In [4]:
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## 1.2) Build your client

Initialize our rush client and fetch available module paths.

In [5]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
)

In [6]:
# |hide
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
    restore_by_default=True
)

# 2) Fetch PDBS

In [7]:
pdb_ids = ["1B39", "1GIH"]
for pdb_id in pdb_ids:
    complex = list(pdb_fetch.fetch_structure(pdb_id))
    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb_id}.pdb", "w") as f:
        for line in complex:
            f.write(line)

# 3) Convert each PDB

In [8]:
for pdb in glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")):
    (conformer,) = await client.convert("PDB", Path(pdb), target="NIX")
    print(await conformer.get())

2024-02-08 08:28:36,986 - rush - INFO - restore=None self.restore_by_default=True
2024-02-08 08:28:36,988 - rush - INFO - Trying to restore job with tags: ['qdx', 'rush-py-v2', 'demo', 'convert'] and path: github:talo/tengu-prelude/10e91b8349fbcf3948956effa91cb918e640cb65#convert
2024-02-08 08:28:37,061 - rush - WARNING - Multiple module instances found with the same tags and path
2024-02-08 08:28:37,119 - rush - INFO - Restoring job from previous run with id ec940fa0-6c2e-4a58-ae0a-3b64fafbf2b1
[{'amino_acid_insertion_codes': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

In [9]:
await client.status()

{}